In [14]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle

In [15]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='fQCxoL3rtyCoV6HZ6kLVqPv6yUklHzv1xH1As_w-ngph',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'carresalevaluepredictiondeploymen-donotdelete-pr-txoi8jcebjevsy'
object_key = 'autos.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3T�RER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [16]:
#print al the different sellers

print(df.seller.value_counts()) #remove the seller type having only 3 cars

df[df.seller != 'gewerblich'] #now all the sellers are same so we can get rid of this column

df=df.drop('seller',1)

#print al the different sellers 
print(df.offerType.value_counts())

df[df.offerType != 'Gesuch']

#remove the Offer Type having only 12 listings df [df.offerType - 'Gesuch'] #now all the offers are same so we can get rid of this column 
df=df.drop('offerType',1)

privat        371525
gewerblich         3
Name: seller, dtype: int64
Angebot    371516
Gesuch         12
Name: offerType, dtype: int64


In [17]:
#Cars having power less than 50ps and above 900ps seems a little suspicious,

#let's remove them and see what we've got now

print(df.shape)

df=df[(df.powerPS > 50) & (df.powerPS < 900)]

print(df.shape) #around 50000 cars ahave been removed which could have inrouduced error to our data

#simlarly, filtering our the cars having registeration years not in the mentioned range #print(df.shape)

df = df[(df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)] 
print(df.shape)

#not much of a difference but still, 10000 rows have been reduced. it's better to #get rid of faulty data instead of keeping them just to increase the size.

(371528, 18)
(319709, 18)
(309171, 18)


In [18]:
#removing irrelevant columns which are either the same for all the cars in teh dataset, or can #introduce bias, so removing them too.

df.drop(['name', 'abtest', 'dateCrawled', 'nrOfPictures', 'lastSeen','postalCode', 'dateCreated'], axis='columns', inplace=True)

In [19]:
#dropping the duplicates from the dataframe and stroing it in a new df. #here all rows having same value in all the mentioned columns will be deleted and by default,
#only first occurance of any such row is kept
new_df = df.copy()
new_df = new_df.drop_duplicates([ 'price', 'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType' ,'notRepairedDamage'])

In [20]:
#As the dataset contained some german words for many features, cahnging them to english
new_df.gearbox.replace(('manuell', 'automatik'), ('manual', 'automatic'), inplace=True)
new_df.fuelType.replace(('benzin', 'andere', 'elektro'), ('petrol', 'others', 'electric'), inplace=True)
new_df.vehicleType.replace(('kleinwagen', 'cabrio', 'kombi', 'andere'), ('small car', 'convertible', 'combination', 'others'), inplace=True)
new_df.notRepairedDamage.replace(('ja', 'nein'), ('Yes', 'No'), inplace=True)

In [21]:
#### Removing the outliers
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]
#Filling NaN values for columns whose data might not be there with the information provider,
# which might lead to some variance but our model
#but we will still be able to give some estimate to the user
new_df['notRepairedDamage'].fillna(value='not-declared', inplace=True)
new_df['fuelType'].fillna(value='not-declared', inplace=True)
new_df['gearbox'].fillna(value='not-declared', inplace=True)
new_df['vehicleType'].fillna(value='not declared', inplace=True)
new_df['model'].fillna(value='not-declared', inplace=True)

In [22]:
new_df.to_csv("autos_preprocesssed.csv")

In [23]:
#Columns which contain categorical values, which we'll need to convert via label encoding
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']
#looping over the labels to do the label encoding for all at once and
#saving the LABEL ENCODING FILES
mapper = {}
for i in labels:
  mapper[i] = LabelEncoder()
  mapper[i].fit(new_df[i])
  tr = mapper[i].transform(new_df[i])
  np.save(str('classes'+i+' .npy'), mapper[i].classes_)
  print(i, ":",mapper[i])
  new_df.loc[:, i + '_labels'] = pd.Series (tr, index=new_df.index)

#Final data to be put in a new dataframe called "LABELED",
labeled = new_df[['price','yearOfRegistration','powerPS','kilometer','monthOfRegistration']+[x+"_labels" for x in labels]]
print(labeled.columns)

gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()
Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [24]:
#Storing price in Y and rest of the datain X
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values
#need to reshape the y values
Y = Y.reshape(-1,1)

In [25]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 3)

In [26]:
#Model building and Fitting
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor (n_estimators=1000, max_depth=15, random_state=34)
#fitting the model
regressor.fit(X_train, np.ravel (Y_train, order='C'))
#predicting the values fo test test
y_pred = regressor.predict(X_test)
#printing the Accuraccy for test set
print(r2_score (Y_test,y_pred))

0.860497492549767


In [67]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
"url": "https://us-south.ml.cloud.ibm.com",
"apikey":""
}
client = APIClient(wml_credentials)

In [71]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [72]:
space_uid = guid_from_space_name(client,'CRVP-deployment')
print("Space UID = "+space_uid)

Space UID = b5599282-5cce-4e1a-ad82-970cc7dc963c


In [73]:
client.set.default_space(space_uid)

'SUCCESS'

In [75]:
client.software_specifications.list(limit = 100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [93]:
software_spec_uid = client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [94]:
model_details = client.repository.store_model(model=regressor,meta_props={
client.repository.ModelMetaNames.NAME:"CRVP-deployment",
client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
)
model_id = client.repository.get_model_id(model_details)

In [95]:
model_id

'ab715b8a-6ef7-4f20-86ec-7f533439e374'